In [1]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import torch
from efficientnet import EfficientNet
from tqdm.notebook import tqdm
import pickle
import os
import PIL
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from torchvision.utils import make_grid, save_image


In [2]:
df_nwk = pd.read_csv('/home/hb/python/pp_pred/raw_data/220531_GPS5.0-NetworKIN/curated.networkin_0111 (1).txt', sep="\t")
df_nwk.head(2)
df_nwk.columns

Index(['#Name', 'Position', 'Tree', 'NetPhorest Group',
       'Kinase/Phosphatase/Phospho-binding domain', 'NetworKIN score',
       'NetPhorest probability', 'STRING score', 'Target STRING ID',
       'Kinase/Phosphatase/Phospho-binding domain STRING ID',
       'Target description',
       'Kinase/Phosphatase/Phospho-binding domain description', 'Target Name',
       'Kinase/Phosphatase/Phospho-binding domain Name',
       'Peptide sequence window', 'Intermediate nodes'],
      dtype='object')

In [13]:
df_gps = pd.read_csv('/home/hb/python/pp_pred/raw_data/220531_GPS5.0-NetworKIN/networkinsub.gps.csv', sep='\t')

In [14]:
df_gps.head(3)

,Position,Code,Kinase,Peptide,Score,Cutoff
0,72,S,AGC,DSTYYKASKKFRTPS,0.601,0.386
1,79,S,AGC,SKKFRTPSFLKKSPG,3.080,0.386
2,122,S,AGC,IPDASRPSSSRSYVL,1.137,0.386


In [ ]:
NGFGTsPLTPS
DSTYYKASKKFRTPS
SKKFRTPSFLKKSPG

In [15]:
df_nwk[['nwk_Kinase_substrate']] = 0
df_gps[['gps_Kinase_substrate']] = 0

In [16]:
df_nwk.head(3)

,#Name,Position,Tree,NetPhorest Group,Kinase/Phosphatase/Phospho-binding domain,NetworKIN score,NetPhorest probability,STRING score,Target STRING ID,Kinase/Phosphatase/Phospho-binding domain STRING ID,Target description,Kinase/Phosphatase/Phospho-binding domain description,Target Name,Kinase/Phosphatase/Phospho-binding domain Name,Peptide sequence window,Intermediate nodes,Kinase_substrate,nwk_Kinase_substrate
0,Q9GZM8,S242,WW,any_group,PIN1,1.1188,0.1258,0.3862,ENSP00000333982,ENSP00000247970,NDEL1,PIN1,nudE nuclear distribution gene E homolog (A. n...,"peptidylprolyl cis/trans isomerase, NIMA-inter...",NGFGTsPLTPS,ENSP00000300093,PIN1_NGFGTsPLTPS,0
1,Q9GZM8,S242,KIN,CDK2_CDK3_CDK1_CDK5_group,CDK1,6.3427,0.1481,0.7380,ENSP00000333982,ENSP00000306043,NDEL1,CDK1,nudE nuclear distribution gene E homolog (A. n...,cyclin-dependent kinase 1,NGFGTsPLTPS,NaN,CDK1_NGFGTsPLTPS,0
2,Q9GZM8,S242,KIN,AuroraA,AuroraA,5.3682,0.0093,0.9410,ENSP00000333982,ENSP00000216911,NDEL1,AURKA,nudE nuclear distribution gene E homolog (A. n...,aurora kinase A,NGFGTsPLTPS,NaN,AURKA_NGFGTsPLTPS,0


In [17]:
df_gps[['peptide']] = 0
df_gps

,Position,Code,Kinase,Peptide,Score,Cutoff,gps_Kinase_substrate,peptide
0,72,S,AGC,DSTYYKASKKFRTPS,0.601,0.386,0,0
1,79,S,AGC,SKKFRTPSFLKKSPG,3.080,0.386,0,0
2,122,S,AGC,IPDASRPSSSRSYVL,1.137,0.386,0,0
3,123,S,AGC,PDASRPSSSRSYVLE,1.638,0.386,0,0
4,134,S,AGC,YVLEKRASGQAFEEA,1.645,0.386,0,0
...,...,...,...,...,...,...,...,...
34778,2575,Y,Dual/Other/WEE/WEE1/WEE1,SKNLIREYVKQTWPK,1.775,1.668,0,0
34779,2749,Y,Dual/Other/WEE/WEE1/WEE1,QAGMMSDYGTARRNA,2.518,1.668,0,0
34780,2859,Y,Dual/Other/WEE/WEE1/WEE1,EELVASLYASSPGGV,3.013,1.668,0,0
34781,2914,Y,Dual/Other/WEE/WEE1/WEE1,PAENFDDYMKEVGQA,2.546,1.668,0,0


In [40]:
df_gps['peptide'] = df_gps['Peptide'].apply(lambda x:x[2:15])

In [19]:
df_nwk['nwk_Kinase_substrate'] = df_nwk['Kinase/Phosphatase/Phospho-binding domain description'].str.cat(df_nwk['Peptide sequence window'], sep='_')
df_nwk.head(3)

,#Name,Position,Tree,NetPhorest Group,Kinase/Phosphatase/Phospho-binding domain,NetworKIN score,NetPhorest probability,STRING score,Target STRING ID,Kinase/Phosphatase/Phospho-binding domain STRING ID,Target description,Kinase/Phosphatase/Phospho-binding domain description,Target Name,Kinase/Phosphatase/Phospho-binding domain Name,Peptide sequence window,Intermediate nodes,Kinase_substrate,nwk_Kinase_substrate
0,Q9GZM8,S242,WW,any_group,PIN1,1.1188,0.1258,0.3862,ENSP00000333982,ENSP00000247970,NDEL1,PIN1,nudE nuclear distribution gene E homolog (A. n...,"peptidylprolyl cis/trans isomerase, NIMA-inter...",NGFGTsPLTPS,ENSP00000300093,PIN1_NGFGTsPLTPS,PIN1_NGFGTsPLTPS
1,Q9GZM8,S242,KIN,CDK2_CDK3_CDK1_CDK5_group,CDK1,6.3427,0.1481,0.7380,ENSP00000333982,ENSP00000306043,NDEL1,CDK1,nudE nuclear distribution gene E homolog (A. n...,cyclin-dependent kinase 1,NGFGTsPLTPS,NaN,CDK1_NGFGTsPLTPS,CDK1_NGFGTsPLTPS
2,Q9GZM8,S242,KIN,AuroraA,AuroraA,5.3682,0.0093,0.9410,ENSP00000333982,ENSP00000216911,NDEL1,AURKA,nudE nuclear distribution gene E homolog (A. n...,aurora kinase A,NGFGTsPLTPS,NaN,AURKA_NGFGTsPLTPS,AURKA_NGFGTsPLTPS


In [23]:
df_gps.head(3)

,Position,Code,Kinase,Peptide,Score,Cutoff,gps_Kinase_substrate,peptide
0,72,S,AGC,TYYKASKKFRTPS,0.601,0.386,0,YKASKKFRTPS
1,79,S,AGC,KFRTPSFLKKSPG,3.080,0.386,0,RTPSFLKKSPG
2,122,S,AGC,DASRPSSSRSYVL,1.137,0.386,0,SRPSSSRSYVL


In [41]:
df_gps['gps_Kinase_substrate'] = df_gps['Kinase'].str.cat(df_gps['peptide'], sep='_')
df_gps.head(3)

,Position,Code,Kinase,Peptide,Score,Cutoff,gps_Kinase_substrate,peptide
0,72,S,AGC,TYYKASKKFRTPS,0.601,0.386,AGC_YKASKKFRTPS,YKASKKFRTPS
1,79,S,AGC,KFRTPSFLKKSPG,3.080,0.386,AGC_RTPSFLKKSPG,RTPSFLKKSPG
2,122,S,AGC,DASRPSSSRSYVL,1.137,0.386,AGC_SRPSSSRSYVL,SRPSSSRSYVL


In [42]:
gps_kinase_substrate = df_gps[['gps_Kinase_substrate']]
nwk_kinase_substrate = df_nwk[['nwk_Kinase_substrate']]

In [43]:
gps_kinase_substrate

,gps_Kinase_substrate
0,AGC_YKASKKFRTPS
1,AGC_RTPSFLKKSPG
2,AGC_SRPSSSRSYVL
3,AGC_RPSSSRSYVLE
4,AGC_KRASGQAFEEA
...,...
34778,Dual/Other/WEE/WEE1/WEE1_IREYVKQTWPK
34779,Dual/Other/WEE/WEE1/WEE1_MSDYGTARRNA
34780,Dual/Other/WEE/WEE1/WEE1_ASLYASSPGGV
34781,Dual/Other/WEE/WEE1/WEE1_FDDYMKEVGQA


In [44]:
gps_kinase_substrate['new'] = gps_kinase_substrate['gps_Kinase_substrate'].apply(lambda x:x.split('/'))
gps_kinase_substrate['new'] = gps_kinase_substrate['new'].apply(lambda x:x[-1])
gps_kinase_substrate
gps_kinase_substrate = gps_kinase_substrate[['new']]
gps_kinase_substrate

/tmp/ipykernel_200429/4268372441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gps_kinase_substrate['new'] = gps_kinase_substrate['gps_Kinase_substrate'].apply(lambda x:x.split('/'))
/tmp/ipykernel_200429/4268372441.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gps_kinase_substrate['new'] = gps_kinase_substrate['new'].apply(lambda x:x[-1])


,new
0,AGC_YKASKKFRTPS
1,AGC_RTPSFLKKSPG
2,AGC_SRPSSSRSYVL
3,AGC_RPSSSRSYVLE
4,AGC_KRASGQAFEEA
...,...
34778,WEE1_IREYVKQTWPK
34779,WEE1_MSDYGTARRNA
34780,WEE1_ASLYASSPGGV
34781,WEE1_FDDYMKEVGQA


In [48]:
gps_kinase_substrate.columns = ['kin_subseq']
gps_kinase_substrate

,kin_subseq
0,AGC_YKASKKFRTPS
1,AGC_RTPSFLKKSPG
2,AGC_SRPSSSRSYVL
3,AGC_RPSSSRSYVLE
4,AGC_KRASGQAFEEA
...,...
34778,WEE1_IREYVKQTWPK
34779,WEE1_MSDYGTARRNA
34780,WEE1_ASLYASSPGGV
34781,WEE1_FDDYMKEVGQA


In [52]:
nwk_kinase_substrate

,kin_subseq
0,PIN1_NGFGTSPLTPS
1,CDK1_NGFGTSPLTPS
2,AURKA_NGFGTSPLTPS
3,NEK2_NGFGTSPLTPS
4,CDK5_NGFGTSPLTPS
...,...
42574,YWHAQ_AESGKSASTEV
42575,YWHAG_AESGKSASTEV
42576,YWHAB_AESGKSASTEV
42577,YWHAH_AESGKSASTEV


In [53]:
gps_ntk_merge = pd.merge(gps_kinase_substrate, nwk_kinase_substrate, on = "kin_subseq", how='inner')
gps_ntk_merge.head(3)

,kin_subseq
0,PDK1_STRTYSLGSAL
1,PDK1_VPPSPSLSRHS
2,PDK1_LYRSPSMPENL


In [55]:
len(gps_ntk_merge)

277

In [54]:
  gps_ntk_merge.to_csv("/home/hb/python/pp_pred/raw_data/220601_pp-pred_(gps_networKIN_중복데이터비교)-v1.0_HB.csv")

